In [ ]:
!unzip /content/archive.zip


Archive:  /content/archive.zip
  inflating: mnist_test.csv          
  inflating: mnist_train.csv         


In [ ]:
import csv
import numpy as np

Using the raw binary features, implement the streaming Naive Bayes algorithm and classify
the test data.

In [ ]:

import pandas as pd
import numpy as np

class NaiveBayes:
    def __init__(self):
        self.class_priors_ = None
        self.feature_counts_ = None
        self.classes_ = None

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.classes_ = np.unique(y)
        n_classes = len(self.classes_)

        self.class_priors_ = np.zeros(n_classes)
        self.feature_counts_ = np.zeros((n_classes, n_features, X.max()+1))

        for c in self.classes_:
            X_c = X[c == y]
            self.class_priors_[c] = X_c.shape[0] / n_samples
            for j in range(n_features):
                values, counts = np.unique(X_c[:, j], return_counts=True)
                self.feature_counts_[c, j, values] += counts

        return self

    def predict(self, X):
        posteriors = []
        for i, x in enumerate(X):
            # Compute posterior for each class
            posterior = np.zeros(len(self.classes_))
            for c in self.classes_:
                log_likelihood = np.sum(np.log(self.feature_counts_[c, np.arange(len(x)), x]))
                log_prior = np.log(self.class_priors_[c])
                posterior[c] = log_likelihood + log_prior

            # Normalize to obtain probabilities
            posterior = np.exp(posterior - np.max(posterior))
            posterior /= np.sum(posterior)
            posteriors.append(posterior)

        return np.argmax(posteriors, axis=1)

# Load the data
train_data = pd.read_csv("/content/mnist_train.csv").values
test_data = pd.read_csv("/content/mnist_test.csv").values

# Split into features and labels
X_train = train_data[:, 1:]
y_train = train_data[:, 0]
X_test = test_data[:, 1:]
y_test = test_data[:, 0]

# Create and fit the model
clf = NaiveBayes()
clf.fit(X_train, y_train)

# Make predictions on the test data
y_pred = clf.predict(X_test)

# Compute accuracy
accuracy = np.mean(y_pred == y_test)
print(f"Accuracy: {accuracy}")



<ipython-input-22-caad2c2f68ab>:33: RuntimeWarning: divide by zero encountered in log
  log_likelihood = np.sum(np.log(self.feature_counts_[c, np.arange(len(x)), x]))
<ipython-input-22-caad2c2f68ab>:38: RuntimeWarning: invalid value encountered in subtract
  posterior = np.exp(posterior - np.max(posterior))


Accuracy: 0.3988


Project the binary features into a lower dimensional space using a dimensionality reduction
technique of your choice (such as PCA, LDA, t-SNE, etc.), by varying the dimensionality in the
range {50,100,200}, and classify the test data using the same algorithm.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.naive_bayes import MultinomialNB
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
import time

# Load the data
train_data = pd.read_csv('/content/mnist_train.csv')
test_data = pd.read_csv('/content/mnist_test.csv')

# Split features and labels
X_train = train_data.iloc[:, 1:].values
y_train = train_data.iloc[:, 0].values
X_test = test_data.iloc[:, 1:].values
y_test = test_data.iloc[:, 0].values

# Convert pixel values to binary features
X_train[X_train >= 128] = 1
X_train[X_train < 128] = 0
X_test[X_test >= 128] = 1
X_test[X_test < 128] = 0

# Dimensionality reduction using PCA
dims = [50, 100, 200]
for dim in dims:
    pca = PCA(n_components=dim)
    X_train_pca = pca.fit_transform(X_train)
    X_test_pca = pca.transform(X_test)

    # Train and test Naive Bayes classifier
    nb = MultinomialNB()
    start_time = time.time()
    nb.fit(X_train_pca, y_train)
    train_time = time.time() - start_time

    start_time = time.time()
    y_pred = nb.predict(X_test_pca)
    test_time = time.time() - start_time

    accuracy = accuracy_score(y_test, y_pred)
    print("Dimensionality: {}".format(dim))
    print("Accuracy: {}".format(accuracy))
    print("Training time: {:.2f} s".format(train_time))
    print("Testing time: {:.2f} s".format(test_time))
    print("")


/usr/local/lib/python3.9/dist-packages/sklearn/decomposition/_pca.py:642: RuntimeWarning: invalid value encountered in divide
  self.explained_variance_ratio_ = self.explained_variance_ / total_var


Dimensionality: 50
Accuracy: 0.1135
Training time: 0.10 s
Testing time: 0.01 s



/usr/local/lib/python3.9/dist-packages/sklearn/decomposition/_pca.py:642: RuntimeWarning: invalid value encountered in divide
  self.explained_variance_ratio_ = self.explained_variance_ / total_var


Dimensionality: 100
Accuracy: 0.1135
Training time: 0.07 s
Testing time: 0.01 s



/usr/local/lib/python3.9/dist-packages/sklearn/decomposition/_pca.py:642: RuntimeWarning: invalid value encountered in divide
  self.explained_variance_ratio_ = self.explained_variance_ / total_var


Dimensionality: 200
Accuracy: 0.1135
Training time: 0.11 s
Testing time: 0.01 s



In [ ]:
import numpy as np
import pandas as pd
from sklearn.naive_bayes import MultinomialNB
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
import time

# Load the data
train_data = pd.read_csv('/content/mnist_train.csv')
test_data = pd.read_csv('/content/mnist_test.csv')

# Split features and labels
X_train = train_data.iloc[:, 1:].values
y_train = train_data.iloc[:, 0].values
X_test = test_data.iloc[:, 1:].values
y_test = test_data.iloc[:, 0].values

# Convert pixel values to binary features
X_train[X_train >= 128] = 1
X_train[X_train < 128] = 0
X_test[X_test >= 128] = 1
X_test[X_test < 128] = 0

# Dimensionality reduction using PCA
dims = [50, 100, 200]
for dim in dims:
    pca = PCA(n_components=dim)
    X_train_pca = pca.fit_transform(X_train)
    X_test_pca = pca.transform(X_test)

    # Train and test Naive Bayes classifier
    nb = MultinomialNB()
    start_time = time.time()
    nb.fit(X_train_pca, y_train)
    train_time = time.time() - start_time

    start_time = time.time()
    y_pred = nb.predict(X_test_pca)
    test_time = time.time() - start_time

    accuracy = accuracy_score(y_test, y_pred)
    print("Dimensionality: {}".format(dim))
    print("Accuracy: {}".format(accuracy))
    print("Training time: {:.2f} s".format(train_time))
    print("Testing time: {:.2f} s".format(test_time))
    print("")


/usr/local/lib/python3.9/dist-packages/sklearn/decomposition/_pca.py:642: RuntimeWarning: invalid value encountered in divide
  self.explained_variance_ratio_ = self.explained_variance_ / total_var


Dimensionality: 50
Accuracy: 0.1135
Training time: 0.05 s
Testing time: 0.00 s



/usr/local/lib/python3.9/dist-packages/sklearn/decomposition/_pca.py:642: RuntimeWarning: invalid value encountered in divide
  self.explained_variance_ratio_ = self.explained_variance_ / total_var


Dimensionality: 100
Accuracy: 0.1135
Training time: 0.13 s
Testing time: 0.01 s



/usr/local/lib/python3.9/dist-packages/sklearn/decomposition/_pca.py:642: RuntimeWarning: invalid value encountered in divide
  self.explained_variance_ratio_ = self.explained_variance_ / total_var


Dimensionality: 200
Accuracy: 0.1135
Training time: 0.11 s
Testing time: 0.01 s



In [ ]:
import numpy as np
from sklearn.decomposition import PCA
from sklearn.naive_bayes import GaussianNB
import time

# Load the data
train_data = np.loadtxt("/content/mnist_train.csv", delimiter=",", skiprows=1)
test_data = np.loadtxt("/content/mnist_test.csv", delimiter=",", skiprows=1)

# Split the data into features and labels
train_y = train_data[:, 0].astype(int)
train_X = train_data[:, 1:]
test_y = test_data[:, 0].astype(int)
test_X = test_data[:, 1:]

# Preprocess the data
train_X[train_X < 128] = 0
train_X[train_X >= 128] = 1
test_X[test_X < 128] = 0
test_X[test_X >= 128] = 1

# Naive Bayes with Features

def fit_predict(train_X, train_y, test_X, n_components):
    pca = PCA(n_components=n_components)
    train_X_reduced = pca.fit_transform(train_X)
    test_X_reduced = pca.transform(test_X)
    clf = GaussianNB()
    start_time = time.time()
    clf.fit(train_X_reduced, train_y)
    elapsed_time_train = time.time() - start_time
    start_time = time.time()
    predictions = clf.predict(test_X_reduced)
    elapsed_time_test = time.time() - start_time
    accuracy = np.mean(predictions == test_y)
    return accuracy, elapsed_time_train, elapsed_time_test

# Evaluate both approaches for different dimensions

dimensions = [50, 100, 200]

# Naive Bayes with Features
print("Naive Bayes \n")
for dim in dimensions:
    print("Dimension:", dim)
    accuracy, elapsed_time_train, elapsed_time_test = fit_predict(train_X, train_y, test_X, dim)
    print("Accuracy:", accuracy)
    print("Training Time:", elapsed_time_train, "seconds")
    print("Testing Time:", elapsed_time_test, "seconds\n")


Naive Bayes with Features

Dimension: 50
Accuracy: 0.8758
Training Time: 0.08437061309814453 seconds
Testing Time: 0.03549027442932129 seconds

Dimension: 100
Accuracy: 0.8807
Training Time: 0.1913907527923584 seconds
Testing Time: 0.0699911117553711 seconds

Dimension: 200
Accuracy: 0.8848
Training Time: 0.2229290008544922 seconds
Testing Time: 0.11449623107910156 seconds

